# Spark LDA

An example of topic modelling a corpus of texts using Spark ML's LDA.


## Setup

Import libraries, configure debugging, start up a local Spark notebook session.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import org.apache.spark.sql._
import $ivy.`org.apache.spark::spark-mllib:2.4.5`


Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.8.2/almond-spark_2.12-0.8.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.5/spark-sql_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.5/spark-parent_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.10.1/parquet-column-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.5/spark-tags_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.6.7.3/jackson-databind-2.6.7.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.5.5/orc-mapreduce-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/xbean/xbean-asm6-shaded/4.8/xbean-asm6-shaded-4.8.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammo

import $ivy.$                                   // Or use any other 2.x version here

import org.apache.spark.sql._

import $ivy.$                                    


In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2-sources.jar


Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@3c362346

In [4]:
import org.apache.spark.ml.clustering.LDA
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.mllib.linalg.Vector
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.types.IntegerType
import org.apache.spark.sql.functions._

import org.apache.spark.ml.clustering.LDA

import org.apache.spark.ml.feature.RegexTokenizer

import org.apache.spark.ml.feature.StopWordsRemover

import org.apache.spark.ml.feature.CountVectorizer

import org.apache.spark.mllib.linalg.Vector

import scala.collection.mutable.WrappedArray

import org.apache.spark.sql.types.IntegerType

import org.apache.spark.sql.functions._

## Build text corpus

### Download delimited-text data

In [5]:
import scala.io.Source
val url = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/devastation-2010-04-10.cex"

import scala.io.Source

url: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/devastation-2010-04-10.cex"

In [6]:
val lines = Source.fromURL(url).getLines.toVector
val obvDescs = lines.map(ln => { val cols = ln.split("#"); cols(7)}).map(_.toLowerCase).
map(_.replaceAll("[^a-z ]", ""))

lines: scala.package.Vector[String] = Vector(
  "ID#Label#Denomination#Metal#Authority#Mint#Region#ObvType#ObvLegend#ObvPortraitId#RevType#RevLegend#RevPortraitId#StartDate#EndDate",
  "http://numismatics.org/aod/id/7513.81516.2.a#AoD 7513.81516.2.a#none#ar#German Empire#http://www.geonames.org/2861650#none#Facing bust#GENERAL FELDMARSCHALL von HINDENBURG##Nude man wrestling nude giant to the ground#TITANEN KAMPF 1914-1915##1915#1915",
  "http://numismatics.org/aod/id/7513.13525.20.a#AoD 7513.13525.20.a#none#none#German Empire#http://www.geonames.org/2825297#none#Group of spike-helmeted German soldiers running to attack to r.; carrying rifles with bayonets and swords.#1914/15/ FELDZUG/ GEGEN FRANKREICH, RUSSLAND,/ ENGLAND, JAPAN U.S.W.#####1915#1915",
  "http://numismatics.org/aod/id/7513.7155.300.c#AoD 7513.7155.300.c#none#fe#German Weimar Republic#http://www.geonames.org/2867714#none#Head of Schlageter, facing, with laurel branch and torch on each side.#ALBERT . LEO . SCHLAGETER/ in 

### Create `DataFrame` with text corpus

In [7]:
// Create RDD:
val txtRdd = spark.sparkContext.parallelize(obvDescs).zipWithIndex

txtRdd: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[1] at zipWithIndex at cmd6.sc:1

In [8]:
// Import implicits *after* creation of context.
import spark.sqlContext.implicits._

val corpus_df = txtRdd.toDF("corpus", "id")


import spark.sqlContext.implicits._


corpus_df: DataFrame = [corpus: string, id: bigint]

## Tokenize

In [9]:
val tokenizer = new RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(4).setInputCol("corpus").setOutputCol("tokens")
val tokenized_df = tokenizer.transform(corpus_df)


tokenizer: RegexTokenizer = regexTok_017e3abaa9df
tokenized_df: DataFrame = [corpus: string, id: bigint ... 1 more field]

## Filter out stop words

Well, think about a serious stop-word list at some point, but here's the technique.

In [10]:
val stopWords = Array("a", "and", "the", "of")

val remover = new StopWordsRemover().setStopWords(stopWords).setInputCol("tokens").setOutputCol("filtered")
val filtered_df = remover.transform(tokenized_df)





stopWords: Array[String] = Array("a", "and", "the", "of")
remover: StopWordsRemover = stopWords_32e320584743
filtered_df: DataFrame = [corpus: string, id: bigint ... 2 more fields]

### Compute counts of each token for each text


In [11]:
val vectorizer = new CountVectorizer().setInputCol("filtered").setOutputCol("features").setVocabSize(10000).setMinDF(5).fit(filtered_df)
val countVectors = vectorizer.transform(filtered_df).select("id", "features")



flatMap at CountVectorizer.scala:205

1 / 1

count at CountVectorizer.scala:230

1 / 1

top at CountVectorizer.scala:233

1 / 1

vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_83707f81e42c
countVectors: DataFrame = [id: bigint, features: vector]

### Create LDA model

In [12]:
val lda = new LDA().setK(10).setMaxIter(10)
val model = lda.fit(countVectors)

count at LDAOptimizer.scala:419

1 / 1

first at LDAOptimizer.scala:420

1 / 1

isEmpty at LDAOptimizer.scala:450

...

isEmpty at LDAOptimizer.scala:450

1 / 1

treeAggregate at LDAOptimizer.scala:504

1 / 1

lda: LDA = lda_843e5bea29b1
model: org.apache.spark.ml.clustering.LDAModel = lda_843e5bea29b1

### Compute topics from model

Each topic is a set of terms with corresponding weights.


In [13]:
val topics = model.describeTopics
topics.show(false)

+-----+------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                     |termWeights                                                                                                                                                                                                                       |
+-----+------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[1, 2, 12, 21, 11, 3, 23, 33, 41, 15]           |[0.06508165168997954, 0.026391935224544322, 0.023888805007839874, 0.018253076995016077, 0.01

topics: DataFrame = [topic: int, termIndices: array<int> ... 1 more field]

### Label topics

1. Create a new DataFrame with ordered lists ot terms by looking up the term for each term index.
2. Number the rows of this DataFrame so we can join it with the existing topic data.

In [14]:
val topicLabels = topics.select("termIndices").map { case Row(r:  WrappedArray[Integer]) => r.map( i => vectorizer.vocabulary(i) ) }
val labelsNumberedLong = topicLabels.rdd.zipWithIndex.toDF("terms", "topicLong")
val labelsIndexed = labelsNumberedLong.withColumn("topic", $"topicLong".cast(IntegerType)).drop("topicLong")

val topicsWithTerms = labelsIndexed.join(topics, labelsIndexed.col("topic") === topics.col("topic")).drop(labelsIndexed.col("topic"))





topicLabels: Dataset[WrappedArray[String]] = [value: array<string>]
labelsNumberedLong: DataFrame = [terms: array<string>, topicLong: bigint]
labelsIndexed: DataFrame = [terms: array<string>, topic: int]
topicsWithTerms: DataFrame = [terms: array<string>, topic: int ... 2 more fields]

In [15]:
val weightedLabels = topicsWithTerms.withColumn("termsWithWeight", expr("zip_with(terms, termWeights, (t,w) -> concat(t, ' ', w))"))


weightedLabels: DataFrame = [terms: array<string>, topic: int ... 3 more fields]

In [16]:
// Flat view
weightedLabels.select("topic", "termsWithWeight").show(false)
// Exploded view
val explodedTerms = weightedLabels.select(col("*"),explode(col("termsWithWeight"))).select("topic","col")

explodedTerms.show(explodedTerms.count.toInt, false)


run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd15.sc:1

1 / 1

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termsWithWeight                                                                                                                                                                                                                                                                                               |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[with 0.06508165168997954, holding 0.026391935224544322, fig

...

1 / 1

show at cmd15.sc:5

1 / 1

+-----+--------------------------------+
|topic|col                             |
+-----+--------------------------------+
|0    |with 0.06508165168997954        |
|0    |holding 0.026391935224544322    |
|0    |figure 0.023888805007839874     |
|0    |male 0.018253076995016077       |
|0    |soldier 0.017934549847355815    |
|0    |sword 0.016723960697871545      |
|0    |nude 0.016175794839949107       |
|0    |shield 0.015047557931133765     |
|0    |figures 0.014145085748481381    |
|0    |eagle 0.013638224667734436      |
|1    |emerging 0.002851792745865678   |
|1    |masses 0.0028367946679882542    |
|1    |swords 0.0028227967449474126    |
|1    |figures 0.002803576705866512    |
|1    |putting 0.0027886689371302647   |
|1    |peace 0.0027862119241163835     |
|1    |branches 0.002773257001901416   |
|1    |solider 0.0027696233043770406   |
|1    |stars 0.002756480758450899      |
|1    |singing 0.0027374092962779923   |
|2    |behind 0.003915130877979812     |
|2    |male 0.00

explodedTerms: DataFrame = [topic: int, col: string]

### Compute distribution of topics per document

In [17]:
val transformed = model.transform(countVectors)
transformed.printSchema // show(false)
transformed.show(3, false)


root
 |-- id: long (nullable = false)
 |-- features: vector (nullable = true)
 |-- topicDistribution: vector (nullable = true)



show at cmd16.sc:3

1 / 1

+---+---------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                   |topicDistribution                                                                                                                                                                                              |
+---+---------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |(452,[],[])                                                                |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]        

transformed: DataFrame = [id: bigint, features: vector ... 1 more field]

## Syncing Dependencies

If extra dependencies are loaded, via ``import $ivy.`…` `` after the `SparkSession` has been created, you should call `NotebookSparkSession.sync()` for the newly added JARs to be passed to the Spark executors.

Make sure the version of Spark used to start the master and executors matches the one loaded in the notebook session
(via e.g. ``import $ivy.`org.apache.spark::spark-sql:X.Y.Z` ``), and that the machine running the kernel can access / is
accessible from all nodes of the standalone cluster.


In [ ]:
import $ivy.`org.typelevel::cats-core:1.6.0`

NotebookSparkSession.sync() // cats should be available on workers

## Datasets and Dataframes

If you try to create a `Dataset` or a `Dataframe` from some data structure containing a case class and you're getting an `org.apache.spark.sql.AnalysisException: Unable to generate an encoder for inner class ...` when calling `.toDS`/`.toDF`, try the following workaround:

Add `org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)` in the same cell where you define case classes involved.

In [ ]:
/*
import spark.implicits._

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this);

case class Person(id: String, value: Int)

val ds = List(Person("Alice", 42), Person("Bob", 43), Person("Charlie", 44)).toDS
*/

In [19]:
import spark.implicits._

import spark.implicits._

This workaround won't be neccessary anymore in future Spark versions.

### Rich Display of Datasets and Dataframes


Add a displayer

In [18]:
// based on a snippet by Ivan Zaitsev
// https://github.com/almond-sh/almond/issues/180#issuecomment-364711999
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        val str = cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
        if (truncate > 0 && str.length > truncate) {
          // do not show ellipses for strings shorter than 4 characters.
          if (truncate < 4) str.substring(0, truncate)
          else str.substring(0, truncate - 3) + "..."
        } else {
          str
        }
      }: Seq[String]
    }

    publish.html(s"""
      <table class="table">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

defined class RichDF

In [ ]:
ds.toDF.showHTML()

In [20]:
explodedTerms.showHTML()

run at ThreadPoolExecutor.java:1149

1 / 1

take at cmd17.sc:4

1 / 1

topic,col
0,with 0.0650816516...
0,holding 0.0263919...
0,figure 0.02388880...
0,male 0.0182530769...
0,soldier 0.0179345...
0,sword 0.016723960...
0,nude 0.0161757948...
0,shield 0.01504755...
0,figures 0.0141450...
0,eagle 0.013638224...


In [26]:
weightedLabels.select("topic", "termsWithWeight").showHTML(truncate=1000)


run at ThreadPoolExecutor.java:1149

1 / 1

take at cmd17.sc:4

1 / 1

topic,termsWithWeight
0,"[with 0.06508165168997954, holding 0.026391935224544322, figure 0.023888805007839874, male 0.018253076995016077, soldier 0.017934549847355815, sword 0.016723960697871545, nude 0.016175794839949107, shield 0.015047557931133765, figures 0.014145085748481381, eagle 0.013638224667734436]"
1,"[emerging 0.002851792745865678, masses 0.0028367946679882542, swords 0.0028227967449474126, figures 0.002803576705866512, putting 0.0027886689371302647, peace 0.0027862119241163835, branches 0.002773257001901416, solider 0.0027696233043770406, stars 0.002756480758450899, singing 0.0027374092962779923]"
2,"[behind 0.003915130877979812, male 0.0038135002062997703, foreground 0.003723825706792354, draped 0.0037216016085944484, background 0.0036886825326194203, view 0.0036325080567951944, woman 0.0035594671263316064, winged 0.003541054574383623, figure 0.00342550637661154, seated 0.0033595737148543075]"
3,"[marianne 0.01758622747657748, embracing 0.014788490484195033, bust 0.013998892891940427, woman 0.013361208386352198, alsatian 0.012377131419678862, cross 0.009678960107537058, swimming 0.00963252168973576, tearing 0.005784773405906503, iron 0.00538449307205187, wearing 0.004947770781993919]"
4,"[head 0.012292463511224198, rising 0.009149614145788807, statue 0.008891709785367501, lion 0.007559298549189765, liberty 0.0070515601703657615, allies 0.005942970613120755, next 0.0058927358089209785, sides 0.005534581148949547, symbol 0.005475686972380628, between 0.005130222934139014]"
5,"[with 0.03384208782843156, bust 0.023503547456879416, soldier 0.020686054130910334, wearing 0.020684301710301802, soldiers 0.019491807703887233, facing 0.01628564497206784, background 0.016261510576328263, over 0.0157165601597623, french 0.014687284896668523, german 0.014490109998136195]"
6,"[which 0.027154369032011863, inscribed 0.009211715717364498, standing 0.00876206316517024, children 0.007918673748567882, leading 0.007911824430257989, door 0.007703344693252737, into 0.007583191334527911, while 0.0069827193288832575, their 0.006894506706000488, woman 0.006392322441172009]"
7,"[elisabeth 0.005393775439936644, albert 0.005192128927123251, busts 0.00406057591091617, canon 0.003726989394160665, around 0.0027816994595422223, italy 0.00274693309094362, military 0.0027359470416905385, mary 0.00273155817662158, kaiser 0.0027248674018032384, howitzer 0.0027208880680611985]"
8,"[bust 0.16801467728016597, facing 0.07139603926123657, slightly 0.0233586150816539, uniformed 0.020193127121709998, hatted 0.01822769029652685, bearded 0.015051781008580144, three 0.01371476474997133, wearing 0.01207733608680216, turned 0.012056693342943656, women 0.00803873762084297]"
9,"[victory 0.07564482596215899, laurel 0.07128926116239366, holding 0.06940671165507856, sword 0.0671701845358891, wreath 0.06701184423351741, crown 0.06253990268733065, flaming 0.058427453982829516, victors 0.058318376701287394, leading 0.004847977429521989, french 0.004730127797531143]"


In [24]:
transformed.select("id","topicDistribution").showHTML(truncate=1000)

take at cmd17.sc:4

1 / 1

id,topicDistribution
0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]"
1,"[0.03436475061513339,0.03146905246624735,0.031478985621317436,0.03183494001943128,0.03173812270549483,0.035189674162586296,0.0318205456764092,0.031498126203549956,0.7072404524258055,0.033365350104024916]"
2,"[0.010244268851978492,0.009380173283261689,0.009382928019682798,0.009489080031394731,0.009460274074781618,0.9129815840098621,0.00948485473650455,0.00938878702159744,0.010242905604385788,0.009945144366550878]"
3,"[0.7911991994404797,0.010425523590921604,0.010428777147129665,0.010546569598967853,0.010515336375992392,0.011662592047208483,0.010542471995064575,0.010435112507372191,0.011392624821432133,0.12285179247543157]"
4,"[0.011384283500802075,0.010424885125462902,0.010428167266255539,0.010545997267178479,0.010514022522258347,0.011656491456285197,0.010541332092261914,0.010434517786042645,0.01138329896466422,0.9026870040187888]"
5,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]"
6,"[0.01281283967788591,0.011732662168603344,0.011736446970909396,0.011869124815071172,0.011832962021127288,0.2784159972117227,0.011863771947281812,0.011743541329538643,0.6255528704553377,0.012439783402521883]"
7,"[0.03436475061513339,0.03146905246624735,0.031478985621317436,0.03183494001943128,0.03173812270549483,0.035189674162586296,0.0318205456764092,0.031498126203549956,0.7072404524258055,0.033365350104024916]"
8,"[0.017103872708562607,0.015663989961035414,0.015668802943102408,0.015845317035282008,0.015797074810033878,0.017517417940262868,0.015838097053308388,0.015677663053547154,0.8542812703465122,0.016606494148353062]"
9,"[0.012815631778901005,0.011733198885903552,0.011736966152673072,0.011869469371042798,0.011833784207619217,0.8911499607503498,0.011864202726724705,0.011743992207360329,0.01281231913851112,0.012440474780914486]"


In [25]:
explodedTerms.showHTML(explodedTerms.count.toInt, 1000)

run at ThreadPoolExecutor.java:1149

1 / 1

count at cmd24.sc:1

1 / 1

count at cmd24.sc:1

...

run at ThreadPoolExecutor.java:1149

1 / 1

take at cmd17.sc:4

1 / 1

topic,col
0,with 0.06508165168997954
0,holding 0.026391935224544322
0,figure 0.023888805007839874
0,male 0.018253076995016077
0,soldier 0.017934549847355815
0,sword 0.016723960697871545
0,nude 0.016175794839949107
0,shield 0.015047557931133765
0,figures 0.014145085748481381
0,eagle 0.013638224667734436
